In [39]:
import h5py
import numpy as np

In [100]:
import os
# Get the path to the folder containing the .h5 files
folder_path = 'experimental_data\magnitometer_data'

variables = {'V0', 'X1', 'X2', 'X3', 'Y1', 'Y2', 'Y3', 'Z1', 'Z2'}
arrays = {var: np.empty(0) for var in variables}


In [101]:
arrays

{'X3': array([], dtype=float64),
 'Y1': array([], dtype=float64),
 'Y2': array([], dtype=float64),
 'V0': array([], dtype=float64),
 'Z2': array([], dtype=float64),
 'Y3': array([], dtype=float64),
 'X1': array([], dtype=float64),
 'Z1': array([], dtype=float64),
 'X2': array([], dtype=float64)}

In [102]:

# Loop through all the files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is a .h5 file
    if filename.endswith('.h5'):
        # Get the full path to the file
        file_path = os.path.join(folder_path, filename)
        # Process the file
        with h5py.File(file_path, 'r') as f:
            traces = f['data']['traces']
            for group in traces:
                arrays[group] = np.concatenate((arrays[group], traces[group][()]['values']))


In [104]:
len(arrays['V0'])

57600

In [97]:
f = h5py.File(file_path, 'r')
traces = f['data']['traces']
V0 = traces['V0'][()]
X1 = traces['X1'][()]
np.concatenate((arrays['V0'],V0['values']))

array([ 0.01875555, -0.34601   , -0.36695501, ..., -0.37436634,
       -0.38338882, -0.37436634])

TypeError: The DTypes <class 'numpy.dtype[void]'> and <class 'numpy.dtype[float64]'> do not have a common DType. For example they cannot be stored in a single array unless the dtype is `object`.

In [56]:
traces = f['data']['traces']


ValueError: Invalid location identifier (invalid location identifier)

In [35]:
for group in traces:
    print(group)

V0
X1
X2
X3
Y1
Y2
Y3
Z1
Z2


In [54]:
V0 = traces['V0'][()]
V0


ValueError: Invalid location identifier (invalid location identifier)

In [27]:
len(V0)

6400

In [32]:
# Create a new HDF5 file to store the combined data
with h5py.File('combined.h5', 'w') as f_out:

    # Loop through all the files in the folder
    for filename in os.listdir(folder_path):
    # Check if the file is a .h5 file
        if filename.endswith('.h5'):
        # Get the full path to the file
            file_path = os.path.join(folder_path, filename)

            # Open the input HDF5 file
            with h5py.File(file_path, 'r') as f_in:
                print(f'Processing {file_path}...')

                # Loop over the groups in the input file
                for group_name, group in f_in.items():

                    # Create a new group in the output file
                    if group_name not in f_out:
                        f_out.create_group(group_name)

                    # Loop over the datasets in the input group
                    for dataset_name in group:

                        # Get the input dataset and its shape and datatype
                        input_dataset = group[dataset_name]
                        shape = input_dataset.shape
                        dtype = input_dataset.dtype

                        # If the dataset doesn't exist in the output file, create it
                        if dataset_name not in f_out[group_name]:
                            maxshape = (None,) + shape[1:]
                            f_out.create_dataset(f'{group_name}/{dataset_name}', shape=shape, dtype=dtype, maxshape=maxshape, chunks=True)

                        # Append the data to the output dataset
                        output_dataset = f_out[f'{group_name}/{dataset_name}']
                        data = input_dataset[()]
                        output_dataset.resize((output_dataset.shape[0] + data.shape[0],) + output_dataset.shape[1:])
                        output_dataset[-data.shape[0]:] = data

Processing experimental_data\magnitometer_data\2023-04-20_0045_5_triangulation_0.h5...


TypeError: Simple selection can't process (b'Dev1', b'NI_USB_6343', b'pulseblaster_0_clockline2', b'internal', b'None', b'Content-Type: application/json {}', b'Dev1', b'Content-Type: application/json {"__version__": "1.1.0", "AI_range": [-10.0, 10.0], "wait_monitor_supports_wait_completed_events": true, "ports": {"port0": {"num_lines": 32, "supports_buffered": true}, "port1": {"num_lines": 8, "supports_buffered": false}, "port2": {"num_lines": 8, "supports_buffered": false}}, "supports_buffered_AO": true, "num_CI": 4, "min_semiperiod_measurement": 1e-07, "clock_mirror_terminal": null, "clock_terminal": "/Dev1/PFI14", "num_AI": 32, "static_AO": false, "max_AI_multi_chan_rate": 500000.0, "supports_buffered_DO": true, "AI_start_delay_ticks": null, "AI_term": "RSE", "MAX_name": "Dev1", "max_AO_sample_rate": 917431.1926605505, "max_DO_sample_rate": 1000000.0, "supports_simultaneous_AI_sampling": false, "AI_start_delay": 7e-08, "clock_limit": 917431.1926605505, "max_AI_single_chan_rate": 500